In [1]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

import numpy as np
import re

from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import scienceplots

import COMBINE_harmonizer
from COMBINE_harmonizer import cfg
from COMBINE_harmonizer import utils_corr

plt.style.use('nature')
plt.rcParams['legend.frameon'] = False
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

## 01. Init

In [2]:
root_dir = '../'

In [3]:
COMBINE_harmonizer.init(f'{root_dir}/config.yaml')

In [4]:
input_dir = f"{cfg.config['out_dir']}/stats"
out_dir = f"{cfg.config['out_dir']}/stats"

out_figure_dir = f"{out_dir}/figure"

os.makedirs(out_figure_dir, exist_ok=True)

data_dict_filename = f"{root_dir}/{COMBINE_harmonizer.DATA_DICTIONARY_EXCEL}"

In [5]:
df_data_dict_main = COMBINE_harmonizer.load_data_dict(data_dict_filename, COMBINE_harmonizer.SHEET_MAIN)
df_data_dict_followup = COMBINE_harmonizer.load_data_dict(data_dict_filename, COMBINE_harmonizer.SHEET_FOLLOW_UP)
df_data_dict_analysis = COMBINE_harmonizer.load_data_dict(data_dict_filename, COMBINE_harmonizer.SHEET_DERIVED_DATA)

data_dict_main_map = {row[COMBINE_harmonizer.DATA_DICT_VAR_NAME]: row[COMBINE_harmonizer.DATA_DICT_VAR_TYPE] for idx, row in df_data_dict_main.iterrows()}
data_dict_followup_map = {row[COMBINE_harmonizer.DATA_DICT_VAR_NAME]: row[COMBINE_harmonizer.DATA_DICT_VAR_TYPE] for idx, row in df_data_dict_followup.iterrows()}
data_dict_analysis_map = {row[COMBINE_harmonizer.DATA_DICT_VAR_NAME]: row[COMBINE_harmonizer.DATA_DICT_VAR_TYPE] for idx, row in df_data_dict_analysis.iterrows()}


_DATA_DICT_MAP = {
    COMBINE_harmonizer.SHEET_MAIN: data_dict_main_map,
    COMBINE_harmonizer.SHEET_FOLLOW_UP: data_dict_followup_map,
    COMBINE_harmonizer.SHEET_DERIVED_DATA: data_dict_analysis_map,
}

In [6]:

_FILENAME_INFO_DATA_DICT_MAP = {each['name']: _DATA_DICT_MAP[each['data_dict']] for each in COMBINE_harmonizer.FILENAME_INFOS}

_PREFIX_DATA_DICT_MAP = {COMBINE_harmonizer.flatten_filename_prefix(filename): val for filename, val in _FILENAME_INFO_DATA_DICT_MAP.items()}

## Load flatten

In [7]:
filename = f'{input_dir}/zz-merged-flatten-inv.csv'

In [8]:
df_inv = pd.read_csv(filename)

/var/folders/84/vxdpc5mx2cqdr6zvp8p_tygc0000gn/T/ipykernel_18900/136453679.py:1: DtypeWarning: Columns (6,36,40,41,137,169,172,178,181,183,185,188,190,194,202,203,204,205,210,214,215,216,217,222,226,227,228,229,230,234,236,2003,2004,2005,2006,2007,2831,2832,2833,2834,2835,2836,2837,2838,2839,2840,2841,2842,2843,2844,2845,2846,2847,2848,2849,2850,2851,2852,2853,2854,2855,2856,2857,2858,2859,2860,2861,2862,2863,2864,2865,2866,2867,2868,2869,2870,2871,2872,3235,3239,3242,3245,3246,3248,3249,3250,3252,3263,3264,3265,3273,3274,3278,3279,3280,3281,3283,3288,3289,3290,3292,3303,3304,3305,3308,3310,3311,3312,3313,3314,3315,3318,3319,3320,3321,3322,3338,3339,3340,3341,3428,3432,3433,3434,3440,3460,3461,3462,3463,3464,3465,3466,3467,3468,3472,3473,3474,3480,3486,3487,3720,3721,3722,3723,3724,3725,3726,3727,3728,3729,3730,3731,3732,3733,3734,3735,3736,3737,3738,3739,3740,3741,3742,3743,3744,3745,3746,3747,3748,3749,3750,3751,3755,3756,3757,3758,3759,3761,3763,3764,3765,3766,3767,3768,3769,3770,37

In [9]:
columns = list(df_inv.columns)

In [10]:
df_columns = pd.DataFrame({'columns': columns})

In [11]:
df_columns['column_tuple'] = df_columns['columns'].apply(COMBINE_harmonizer.flatten_column_tuple)
df_columns['column_name'] = df_columns['column_tuple'].apply(lambda x: x[1])
df_columns['column_prefix'] = df_columns['column_tuple'].apply(lambda x: x[0])

In [12]:
df_columns['column_prefix'].unique()

array(['', '01-02', '01-03', '01-04', '01-05', '01-06', '01-07', '01-08',
       '01-09', '01-10', '01-11', '01-12', '02-01', '02-02', '02-03',
       '02-04', '02-05', '02-05_s', '02-06_s', '02-07', '02-08', '02-09',
       '02-11', '02-12', '02-13', '02-14', '02-15', '02-16', '02-17',
       '03-01', '03-01_s', '03-02', '03-03', '03-04', '03-05', '03-05_s',
       '04-01', '04-03', '04-04', '04-05', '04-06', '04-07', '04-08',
       '04-09', '04-10', '04-11', '04-12', '04-02', '04-13', '04-14',
       '04-15', '04-16', '04-17', '20-00', '20-01', '20-02', '20-03',
       '20-04', '20-05', '20-06', '20-07', '20-08', '30-01', '30-02',
       '30-03', '31-02', '31-03', '31-04', '31-05', '31-06', '31-07'],
      dtype=object)

In [13]:
columns = list(df_inv.columns)
valid_columns = []
for idx, column in tqdm(enumerate(columns)):
    prefix, var_name, postfix = COMBINE_harmonizer.flatten_column_tuple(column)
    if prefix not in _PREFIX_DATA_DICT_MAP:
        print(f'[WARN] ({idx}/{len(columns)}) not in _PREFIX_DATA_DICT_MAP: column: {column} prefix: {prefix}')
        continue

    data_dict_map = _PREFIX_DATA_DICT_MAP[prefix]
    if var_name not in data_dict_map:
        print(f'[WARN] ({idx}/{len(columns)}) not in DATA_DICT_MAP: {var_name}')
        continue
    valid_columns.append(column)

df_inv_valid = df_inv[valid_columns]

0it [00:00, ?it/s]

6131it [00:00, 1424511.29it/s]

[WARN] (0/6131) not in _PREFIX_DATA_DICT_MAP: column: _study prefix: 
[WARN] (1/6131) not in _PREFIX_DATA_DICT_MAP: column: center prefix: 
[WARN] (2/6131) not in _PREFIX_DATA_DICT_MAP: column: subjectID prefix: 
[WARN] (3/6131) not in _PREFIX_DATA_DICT_MAP: column: uniqueID prefix: 
[WARN] (5307/6131) not in DATA_DICT_MAP: MRI_ID
[WARN] (5308/6131) not in DATA_DICT_MAP: MRI_ID
[WARN] (5309/6131) not in DATA_DICT_MAP: MRI_ID
[WARN] (5499/6131) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (5507/6131) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (5584/6131) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (5651/6131) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (5749/6131) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (5793/6131) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (5798/6131) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (5809/6131) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (5910/6131) not in DATA_DICT_MAP: uniqueFollowupID
[WARN] (6117/6131) not in DATA_DICT_MAP: cordMish

In [14]:
len(df_inv_valid.columns), list(df_inv_valid.columns)

(6106,
 ['01-02:siteID',
  '01-02:birthNumber',
  '01-02:screenComment',
  '01-02:coreTempLess32p5COverEq2Hr_e',
  '01-02:coreTempLess33p5COver1Hr_e',
  '01-02:coreTempLess34COver1Hr_e',
  '01-02:first6HrCoolByClinicalProtocol_e',
  '01-02:chromosomalAbnormality_e',
  '01-02:majorCongenitalAnomaly_e',
  '01-02:birthWeightLessEq1800g_e',
  '01-02:infantUnlikelySurvive_e',
  '01-02:first60MinAllBloodGasPHGreater7p15BaseDeficitLess10mEqPerL_e',
  '01-02:postnatalAgeLess6HrOrGreater24Hr_e',
  '01-02:enrolledConflictingTrial_e',
  '01-02:first60MinAnyBloodGasPHLessEq7_i',
  '01-02:first60MinAnyBloodGasBaseDeficitGreaterEq16mEqPerL_i',
  '01-02:historyPerinatalEvent_i',
  '01-02:at10MinApgarLessEq5OrVent_i',
  '01-02:randomEligible',
  '01-02:consentStatus',
  '01-02:noConsentReason',
  '01-02:noInStudyReason',
  '01-02:random',
  '01-02:noRandomReason',
  '01-02:noRandomReasonText',
  '01-02:randomDate',
  '01-02:randomTime',
  '01-02:randomNumber',
  '01-02:randomTreatmentAssign',
  '01-02

## 03. all

In [15]:
y_columns = [
    '30-02:moderateSevereDisabilityOrDeath',
    '30-02:disabilityLevel',
    '31-05:disabilityLevelDeath',

    '20-04:BayleyIIICognitiveComposite',
    '20-04:BayleyIIILanguageComposite',
    '20-04:BayleyIIIMotorComposite',
]

value_columns = [
    'corr',
    'r2',
    'pvalue',
    'valid',
    'valid_x',
    'valid_y',
    'valid_percent',
]

def _reorder_columns(y_columns, value_columns):
    for y_column in y_columns:
        for value_column in value_columns:
            yield y_column, value_column

reorder_columns = list(_reorder_columns(y_columns, value_columns))

In [16]:
_EXCLUDE_COLUMNS = y_columns + [
    '01-01:birthNumber', # 1: 525 2: 7
    '01-01:coreTempBelow32p5COver2H', # 0: 532 (fit criteria),
    '01-07:pre_CoolbyIceGelPack', # 0: 250, 1: 2

    '30-01:blindness', # part of the outcome definition
    '30-01:multipleImpairment', # part of the outcome defition
    '30-01:moderateSevereCerebralPalsy', # part of the outcome defition
    '30-01:hearingImpairedWithAid', # part of the outcome defition
    '30-01:cerebralPalsy', # part of the outcome defition
    '30-01:afterDischargeSeizure', # part of the outcome defition
    '30-01:grossMotorFunctionLevel', # part of the outcome defition

    '30-01:ageDeath_day', # related t the outcome definition

    '30-02:normalPrimaryOutcome', # part of the outcome defition
    '30-02:moderateSevereDisabilitySurvivor', # part of the outcome defition
    '30-02:disabilityLevelSurvivor', # part of the outcome defition
    '30-02:disabilityLevel', # part of the outcome defition
    '30-02:outcomeGroup', # part of the outcome defition
]

_INCLUDE_COLUMN_PREFIX = [
    '01-02',
    '01-03',
    '01-04',
    '01-05',
    '01-06',
    '01-07',
    '01-08',
    '01-09',
    '01-10',
    '01-12',
    '03-04',
    '03-05',
    '04-01',
    '04-02',
    '04-03',
    '04-04',
    '04-05',
    '04-06',
    '04-07',
    '04-08',
    '04-09',
    '04-10',
    '04-11',
    '04-12',
    '04-13',
    '04-14',
    '04-15',
    '30-01',
    '30-02',
    '30-03',
    '31-02',
    '31-03',
    '31-04',
    '31-05',
    '31-06',
    '31-07',
]


In [17]:
def _valid_column(df, column):
    if column in _EXCLUDE_COLUMNS:
        return False

    column_prefix, var_name, postfix = COMBINE_harmonizer.flatten_column_tuple(column)
    if column_prefix not in _INCLUDE_COLUMN_PREFIX:
        return

    if 'Bayley' in column:
        return False

    if 'death' in column:
        return False

    if 'wdrawSupport' in column:
        return False

    if 'limitCare' in column:
        return False

    if re.search(r':followupCenter$', column):
        return False

    # XXX remove Date
    if re.search(r'Date\d*$', column):
        return False

    # XXX remove Time, but retain spontaneousRespirationTime
    if re.search(r'Time\d*$', column) and 'spontaneousRespirationTime' not in column:
        return False

    # with only MRINRNPatternOfInjury in MRI.
    if column_prefix == '03-05':
        if var_name != 'MRINRNPatternOfInjury':
            return False

    if df[column].dtype != np.float64:
        print(f'[INFO] invalid dtype: column: {column} dtype: {df[column].dtype}')
        return False

    if (df[column].isnull() == False).sum() < 20:
        print(f'[INFO] to few valid samples: column: {column} count: {(df[column].isnull() == False).sum()}')
        return False

    if (df[column].sem() == 0):
        print(f'[INFO] constant column: {column}')
        return False

    return True

In [18]:
x_columns = list(filter(lambda x: _valid_column(df_inv, x), df_inv.columns))
len(x_columns)

[INFO] invalid dtype: column: 01-02:siteID dtype: object
[INFO] invalid dtype: column: 01-02:screenComment dtype: object
[INFO] constant column: 01-02:coreTempLess32p5COverEq2Hr_e
[INFO] constant column: 01-02:coreTempLess33p5COver1Hr_e
[INFO] constant column: 01-02:coreTempLess34COver1Hr_e
[INFO] constant column: 01-02:first6HrCoolByClinicalProtocol_e
[INFO] constant column: 01-02:chromosomalAbnormality_e
[INFO] constant column: 01-02:majorCongenitalAnomaly_e
[INFO] constant column: 01-02:birthWeightLessEq1800g_e
[INFO] constant column: 01-02:infantUnlikelySurvive_e
[INFO] constant column: 01-02:first60MinAllBloodGasPHGreater7p15BaseDeficitLess10mEqPerL_e
[INFO] constant column: 01-02:postnatalAgeLess6HrOrGreater24Hr_e
[INFO] constant column: 01-02:enrolledConflictingTrial_e
[INFO] constant column: 01-02:historyPerinatalEvent_i
[INFO] constant column: 01-02:at10MinApgarLessEq5OrVent_i
[INFO] constant column: 01-02:randomEligible
[INFO] invalid dtype: column: 01-02:consentStatus dtype:

329

In [19]:
x_columns

['01-02:birthNumber',
 '01-02:first60MinAnyBloodGasPHLessEq7_i',
 '01-02:first60MinAnyBloodGasBaseDeficitGreaterEq16mEqPerL_i',
 '01-02:treatmentBlanketType',
 '01-02:inOtherTrial',
 '01-03:motherAge_year',
 '01-03:motherEducation',
 '01-03:motherInsurance',
 '01-04:gravida',
 '01-04:parity',
 '01-04:multipleBirth',
 '01-04:prenatalCare',
 '01-04:hypertensionEclampsia',
 '01-04:antepartumHemorrhage',
 '01-04:thyroidMalfunction',
 '01-04:diabetes',
 '01-05:ruptureOver18Hr',
 '01-05:ruptureBeforeDelivery',
 '01-05:labor',
 '01-05:fetalDecelerate',
 '01-05:cordMishap',
 '01-05:uterineRupture',
 '01-05:shoulderDystocia',
 '01-05:placentalProblem',
 '01-05:maternalHemorrhage',
 '01-05:maternalTrauma',
 '01-05:maternalCardioRespiratoryArrest',
 '01-05:maternalSeizure',
 '01-05:pyrexiaOver37p6C',
 '01-05:chorioamnionitis',
 '01-05:placentalPathologyPerformed',
 '01-05:histologicChorioamionitis',
 '01-05:laborAntibiotics',
 '01-06:encephalopathyLevel',
 '01-06:randomInfantAge',
 '01-06:birthWe

In [20]:
df_corr_all = COMBINE_harmonizer.corr(df_inv, x_columns, y_columns)

df_corr_all_pivot = pd.pivot_table(df_corr_all, index='x', columns='y', values=value_columns).transpose().swaplevel().transpose()

df_corr_all_pivot[('pvalue_lt_threshold', '')] = df_corr_all_pivot[('20-04:BayleyIIICognitiveComposite', 'pvalue')] < COMBINE_harmonizer.P_VALUE_THRESHOLD

all_reorder_columns = [('pvalue_lt_threshold', '')] + reorder_columns
df_corr_all_pivot = df_corr_all_pivot[all_reorder_columns].sort_values(by=[('pvalue_lt_threshold', ''), ('20-04:BayleyIIICognitiveComposite', 'r2')], ascending=[False, False]).reset_index(drop=False)
df_corr_all_pivot

out_filename = f'{out_dir}/31-02-corr-all.csv'
df_corr_all_pivot.to_csv(out_filename, index=False)

  0%|          | 0/329 [00:00<?, ?it/s]

  2%|▏         | 5/329 [00:00<00:07, 44.87it/s]

_corr_coeff: to eval: x: 01-02:birthNumber y: 30-02:moderateSevereDisabilityOrDeath valid_x: 532 valid_y: 504
_corr_coeff: done: x: 01-02:birthNumber y: 30-02:moderateSevereDisabilityOrDeath
_corr_coeff: to eval: x: 01-02:birthNumber y: 30-02:disabilityLevel valid_x: 532 valid_y: 498
_corr_coeff: done: x: 01-02:birthNumber y: 30-02:disabilityLevel
_corr_coeff: to eval: x: 01-02:birthNumber y: 31-05:disabilityLevelDeath valid_x: 532 valid_y: 498
_corr_coeff: done: x: 01-02:birthNumber y: 31-05:disabilityLevelDeath
_corr_coeff: to eval: x: 01-02:birthNumber y: 20-04:BayleyIIICognitiveComposite valid_x: 532 valid_y: 421
_corr_coeff: done: x: 01-02:birthNumber y: 20-04:BayleyIIICognitiveComposite
_corr_coeff: to eval: x: 01-02:birthNumber y: 20-04:BayleyIIILanguageComposite valid_x: 532 valid_y: 411
_corr_coeff: done: x: 01-02:birthNumber y: 20-04:BayleyIIILanguageComposite
_corr_coeff: to eval: x: 01-02:birthNumber y: 20-04:BayleyIIIMotorComposite valid_x: 532 valid_y: 415
_corr_coeff: do

_corr_coeff: to eval: x: 01-03:motherAge_year y: 20-04:BayleyIIICognitiveComposite valid_x: 532 valid_y: 421
_corr_coeff: done: x: 01-03:motherAge_year y: 20-04:BayleyIIICognitiveComposite
_corr_coeff: to eval: x: 01-03:motherAge_year y: 20-04:BayleyIIILanguageComposite valid_x: 532 valid_y: 411
_corr_coeff: done: x: 01-03:motherAge_year y: 20-04:BayleyIIILanguageComposite
_corr_coeff: to eval: x: 01-03:motherAge_year y: 20-04:BayleyIIIMotorComposite valid_x: 532 valid_y: 415
_corr_coeff: done: x: 01-03:motherAge_year y: 20-04:BayleyIIIMotorComposite
_corr_coeff: to eval: x: 01-03:motherEducation y: 30-02:moderateSevereDisabilityOrDeath valid_x: 450 valid_y: 504
_corr_coeff: done: x: 01-03:motherEducation y: 30-02:moderateSevereDisabilityOrDeath
_corr_coeff: to eval: x: 01-03:motherEducation y: 30-02:disabilityLevel valid_x: 450 valid_y: 498
_corr_coeff: done: x: 01-03:motherEducation y: 30-02:disabilityLevel
_corr_coeff: to eval: x: 01-03:motherEducation y: 31-05:disabilityLevelDeath 

_corr_coeff: to eval: x: 01-03:motherInsurance y: 30-02:disabilityLevel valid_x: 359 valid_y: 498
_corr_coeff: done: x: 01-03:motherInsurance y: 30-02:disabilityLevel
_corr_coeff: to eval: x: 01-03:motherInsurance y: 31-05:disabilityLevelDeath valid_x: 359 valid_y: 498
_corr_coeff: done: x: 01-03:motherInsurance y: 31-05:disabilityLevelDeath
_corr_coeff: to eval: x: 01-03:motherInsurance y: 20-04:BayleyIIICognitiveComposite valid_x: 359 valid_y: 421
_corr_coeff: done: x: 01-03:motherInsurance y: 20-04:BayleyIIICognitiveComposite
_corr_coeff: to eval: x: 01-03:motherInsurance y: 20-04:BayleyIIILanguageComposite valid_x: 359 valid_y: 411
_corr_coeff: done: x: 01-03:motherInsurance y: 20-04:BayleyIIILanguageComposite
_corr_coeff: to eval: x: 01-03:motherInsurance y: 20-04:BayleyIIIMotorComposite valid_x: 359 valid_y: 415
_corr_coeff: done: x: 01-03:motherInsurance y: 20-04:BayleyIIIMotorComposite
_corr_coeff: to eval: x: 01-04:gravida y: 30-02:moderateSevereDisabilityOrDeath valid_x: 532 

...

_corr_coeff: to eval: x: 31-02:dischargeTotalModifiedSarnatScore y: 20-04:BayleyIIICognitiveComposite valid_x: 490 valid_y: 421
_corr_coeff: done: x: 31-02:dischargeTotalModifiedSarnatScore y: 20-04:BayleyIIICognitiveComposite
_corr_coeff: to eval: x: 31-02:dischargeTotalModifiedSarnatScore y: 20-04:BayleyIIILanguageComposite valid_x: 490 valid_y: 411
_corr_coeff: done: x: 31-02:dischargeTotalModifiedSarnatScore y: 20-04:BayleyIIILanguageComposite
_corr_coeff: to eval: x: 31-02:dischargeTotalModifiedSarnatScore y: 20-04:BayleyIIIMotorComposite valid_x: 490 valid_y: 415
_corr_coeff: done: x: 31-02:dischargeTotalModifiedSarnatScore y: 20-04:BayleyIIIMotorComposite
_corr_coeff: to eval: x: 31-03:MRINRNPatternOfInjuryMerge y: 30-02:moderateSevereDisabilityOrDeath valid_x: 458 valid_y: 504
_corr_coeff: done: x: 31-03:MRINRNPatternOfInjuryMerge y: 30-02:moderateSevereDisabilityOrDeath
_corr_coeff: to eval: x: 31-03:MRINRNPatternOfInjuryMerge y: 30-02:disabilityLevel valid_x: 458 valid_y: 498

 99%|█████████▉| 325/329 [00:06<00:00, 47.10it/s]

_corr_coeff: to eval: x: 31-03:MRINRNPatternOfInjuryAvg y: 30-02:moderateSevereDisabilityOrDeath valid_x: 441 valid_y: 504
_corr_coeff: done: x: 31-03:MRINRNPatternOfInjuryAvg y: 30-02:moderateSevereDisabilityOrDeath
_corr_coeff: to eval: x: 31-03:MRINRNPatternOfInjuryAvg y: 30-02:disabilityLevel valid_x: 441 valid_y: 498
_corr_coeff: done: x: 31-03:MRINRNPatternOfInjuryAvg y: 30-02:disabilityLevel
_corr_coeff: to eval: x: 31-03:MRINRNPatternOfInjuryAvg y: 31-05:disabilityLevelDeath valid_x: 441 valid_y: 498
_corr_coeff: done: x: 31-03:MRINRNPatternOfInjuryAvg y: 31-05:disabilityLevelDeath
_corr_coeff: to eval: x: 31-03:MRINRNPatternOfInjuryAvg y: 20-04:BayleyIIICognitiveComposite valid_x: 441 valid_y: 421
_corr_coeff: done: x: 31-03:MRINRNPatternOfInjuryAvg y: 20-04:BayleyIIICognitiveComposite
_corr_coeff: to eval: x: 31-03:MRINRNPatternOfInjuryAvg y: 20-04:BayleyIIILanguageComposite valid_x: 441 valid_y: 411
_corr_coeff: done: x: 31-03:MRINRNPatternOfInjuryAvg y: 20-04:BayleyIIILangu

100%|██████████| 329/329 [00:06<00:00, 51.25it/s]

_corr_coeff: to eval: x: 31-04:perinatalSentinelEvent y: 20-04:BayleyIIICognitiveComposite valid_x: 532 valid_y: 421
_corr_coeff: done: x: 31-04:perinatalSentinelEvent y: 20-04:BayleyIIICognitiveComposite
_corr_coeff: to eval: x: 31-04:perinatalSentinelEvent y: 20-04:BayleyIIILanguageComposite valid_x: 532 valid_y: 411
_corr_coeff: done: x: 31-04:perinatalSentinelEvent y: 20-04:BayleyIIILanguageComposite
_corr_coeff: to eval: x: 31-04:perinatalSentinelEvent y: 20-04:BayleyIIIMotorComposite valid_x: 532 valid_y: 415
_corr_coeff: done: x: 31-04:perinatalSentinelEvent y: 20-04:BayleyIIIMotorComposite
_corr_coeff: to eval: x: 31-06:emergencyCSection y: 30-02:moderateSevereDisabilityOrDeath valid_x: 532 valid_y: 504
_corr_coeff: done: x: 31-06:emergencyCSection y: 30-02:moderateSevereDisabilityOrDeath
_corr_coeff: to eval: x: 31-06:emergencyCSection y: 30-02:disabilityLevel valid_x: 532 valid_y: 498
_corr_coeff: done: x: 31-06:emergencyCSection y: 30-02:disabilityLevel
_corr_coeff: to eval:

In [21]:
df_corr_all_pivot

y                                        x pvalue_lt_threshold  \
                                                                 
0                    30-01:gastrostomyTube                True   
1                                30-03:BGT                True   
2                             30-03:extent                True   
3            03-05:MRINRNPatternOfInjury:1                True   
4                               30-03:PLIC                True   
..                                     ...                 ...   
324                     04-05:dischargeDIC               False   
325            01-06:at10MinBaggingAndMask               False   
326  04-13:dischargeSeizurePreIntervention               False   
327                04-07:dischargeDialysis               False   
328    30-03:lateralHemisphericDevastation               False   

y   30-02:moderateSevereDisabilityOrDeath                                 \
                                     corr        r2        pvalue  valid   
0                                0.485146  0.235366  3.569956e-18  284.0   
1                                0.649612  0.421995  4.141362e-37  298.0   
2                                0.528702  0.279526  3.120181e-11  137.0   
3                                0.549684  0.302152  5.320614e-28  337.0   
4                                0.597946  0.357539  2.805184e-30  298.0   
..                                    ...       ...           ...    ...   
324                              0.228704  0.052306  2.096174e-07  504.0   
325                              0.016057  0.000258  7.458231e-01  410.0   
326                             -0.074098  0.005490  2.083478e-01  290.0   
327                              0.068173  0.004648  1.263966e-01  504.0   
328                             -0.079606  0.006337  6.649477e-01   32.0   

y                                 30-02:disabilityLevel  ...  \
    valid_x valid_y valid_percent                  corr  ...   
0     287.0   504.0      0.533835              0.481122  ...   
1     298.0   504.0      0.560150              0.650111  ...   
2     137.0   504.0      0.257519              0.510133  ...   
3     355.0   504.0      0.633459              0.551901  ...   
4     298.0   504.0      0.560150              0.596209  ...   
..      ...     ...           ...                   ...  ...   
324   532.0   504.0      0.947368              0.221352  ...   
325   430.0   504.0      0.770677              0.005861  ...   
326   305.0   504.0      0.545113             -0.087448  ...   
327   532.0   504.0      0.947368              0.066167  ...   
328    32.0   504.0      0.060150             -0.077960  ...   

y   20-04:BayleyIIILanguageComposite                        \
                             valid_x valid_y valid_percent   
0                              287.0   411.0      0.513158   
1                              298.0   411.0      0.494361   
2                              137.0   411.0      0.231203   
3                              355.0   411.0      0.554511   
4                              298.0   411.0      0.494361   
..                               ...     ...           ...   
324                            532.0   411.0      0.772556   
325                            430.0   411.0      0.612782   
326                            305.0   411.0      0.421053   
327                            532.0   411.0      0.772556   
328                             32.0   411.0      0.031955   

y   20-04:BayleyIIIMotorComposite                                         \
                             corr        r2        pvalue  valid valid_x   
0                       -0.509365  0.259452  1.465602e-19  275.0   287.0   
1                       -0.627301  0.393507  2.153775e-30  265.0   298.0   
2                       -0.574763  0.330353  1.952502e-12  126.0   137.0   
3                       -0.504889  0.254912  8.241853e-21  300.0   355.0   
4                       -0.575369  0.331050  9.281420e-2

## 04. Plot

In [22]:
def _plot_corr(df, x_column, y_column, x_column_info=None, y_column_info=None, is_close=False):
    fig = COMBINE_harmonizer.plot_corr(df, x_column, y_column, x_column_info, y_column_info)

    x_str = re.sub(r':', '_', x_column)
    y_str = re.sub(r':', '_', y_column)

    fig.savefig(f'{out_figure_dir}/31-{x_str}-{y_str}-corr.pdf', bbox_inches='tight')
    fig.savefig(f'{out_figure_dir}/31-{x_str}-{y_str}-corr.png', bbox_inches='tight')
    fig.savefig(f'{out_figure_dir}/31-{x_str}-{y_str}-corr.eps', bbox_inches='tight')
    if is_close:
        plt.close(fig)


In [23]:
y_columns_map = {
    '20-04:BayleyIIICognitiveComposite': {'title': 'Bayley III Cognitive Composite Score', 'unit': ''},
    # '20-04:BayleyIIILanguageComposite': {'title': 'Bayley III Language Composite Score', 'unit': ''},
    # '20-04:BayleyIIIMotorComposite': {'title': 'Bayley III Motor Composite Score', 'unit': ''},

    # '30-02:moderateSevereDisabilityOrDeath': {'title': 'Death or Moderate/Severe Disability', 'unit': ''},

    # '30-02:disabilityLevel': {'title': 'Disability Level', 'unit': ''},

    '31-05:disabilityLevelDeath': {'title': 'Disability Level', 'unit': ''},
}

x_columns_map = {
    # '01-06:firstPostnatalBloodGasPH': {'title': 'Blood Gas pH within 1st HR', 'unit': ''},

    # '04-04:dischargeChronicLungDisease': {'title': 'Discharge - Chronic Lung Disease', 'unit': ''},

    # '30-01:gastrostomyTube': {'title': 'Gastrostomy Tube', 'unit': ''},

    # '04-08:dischargeFullNippleFeed': {'title': 'Discharge - Full Nipple Feed', 'unit': ''},

    # '01-06:Apgar5min': {'title': 'Apgar Score at 5 min', 'unit': ''},

    # '01-06:Apgar10min': {'title': 'Apgar Score at 10 min', 'unit': ''},

    # '01-06:encephalopathyLevel': {'title': 'Encephalophathy Level', 'unit': ''},

    # '30-01:bloodGasPH': {'title': 'Blood Gas pH', 'unit': ''},

    # '01-06:cordBloodGasPH': {'title': 'Cord Blood Gas pH', 'unit': ''},

    # '31-03:MRINRNPatternOfInjuryMerge': {'title': 'NRN Pattern of Injury', 'unit': ''},
    # '31-03:MRINRNPatternOfInjuryAvg': {'title': 'NRN Pattern of Injury - Mean', 'unit': ''},
    # '31-03:MRINRNPatternOfInjuryMax': {'title': 'NRN Pattern of Injury - Max', 'unit': ''},

    # '30-03:MRI2LevelPatternOfInjury': {'title': 'NRN Pattern of Injury - 2 Level', 'unit': ''},
    # '30-03:MRINRNPatternOfInjury': {'title': 'NRN Pattern of Injury', 'unit': ''},

    # '31-02:pre_TotalModifiedSarnatScore': {'title': 'Total Modified Sarnat Score - Screening', 'unit': ''},
    # '31-02:post_TotalModifiedSarnatScore': {'title': 'Total Modified Sarnat Score - Post-treatment', 'unit': ''},
    # '31-02:dischargeTotalModifiedSarnatScore': {'title': 'Total Modified Sarnat Score - Discharge', 'unit': ''},

    # '01-12:pre_NeuroExamSeizure': {'title': 'Seizure - Screening', 'unit': ''},

    # '30-01:Apgar10minLte5': {'title': 'Apgar Score at 10 min $\leq$ 5', 'unit': ''},

    # '30-01:Apgar10minLt5': {'title': 'Apgar Score at 10 min < 5', 'unit': ''},

    # '01-06:infantSex': {'title': 'Infant Sex', 'unit': ''},
    # '30-01:maleSex': {'title': 'Infant Sex', 'unit': ''},

    # '01-06:birthGestationalAge_week': {'title': 'Birth Gestational Age', 'unit': 'Week'},
    # '01-03:motherEducation': {'title': 'Mother Education', 'unit': ''},
    # '01-03:motherAge_year': {'title': 'Mother Age', 'unit': 'Year'},

    # '01-06:birthWeight_g': {'title': 'Birth Weight', 'unit': 'g'},

    '31-07:lengthOfStay_day': {'title': 'Length of Stay', 'unit': 'Day'},
}

for x_column, x_column_info in x_columns_map.items():
    for y_column, y_column_info in y_columns_map.items():
        _plot_corr(df_inv, x_column, y_column, x_column_info, y_column_info, is_close=True)